In [1]:
# !pip install -qU python-gdcm pydicom pylibjpeg
# !pip install -U pylibjpeg-libjpeg -v
# !pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg
# !pip install pydicom

# import include nesscessaryz
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil


# Processing data RSNA

In [2]:
meta_rsna = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\CSV_MAIN\rsna.csv')
meta = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Collect_Data\rsna.csv')

meta_rsna = meta_rsna[["patient_id"	,"image_id",	"view"	,"laterality"	,"age"	,"cancer"]]

meta_rsna

,patient_id,image_id,view,laterality,age,cancer
0,10006,1874946579,CC,R,61.0,0
1,10006,1864590858,MLO,R,61.0,0
2,10006,462822612,CC,L,61.0,0
3,10006,1459541791,MLO,L,61.0,0
4,10011,541722628,CC,R,55.0,0
...,...,...,...,...,...,...
50171,9973,1703611570,MLO,L,43.0,0
50172,9989,439796429,CC,R,60.0,0
50173,9989,398038886,MLO,R,60.0,0
50174,9989,1078943060,CC,L,60.0,0


In [3]:
check_cancer = {}

In [4]:
# creat dict cancer
for i in  range(len(meta)):
    # print(meta_rsna)
    check_cancer.update({meta.image_id[i]: meta.cancer[i]})

In [5]:
fake = []
for i in range(len(meta_rsna)):
    if meta_rsna.cancer[i] != check_cancer[meta_rsna.image_id[i]]:
        fake.append(1)
        # print(meta_rsna[i])
    else:
        fake.append(0)

In [6]:
meta_rsna['Fake'] = fake

In [7]:
meta_rsna['Fake'].value_counts()

0    50056
1      120
Name: Fake, dtype: int64

In [10]:
from PIL import Image

# Tạo ảnh mới với kích thước 512x1024 và kiểu 'L' (8-bit grayscale)
img = np.zeros((1024, 512), dtype=np.uint8)
imageio.imwrite("black.png", img)

In [14]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [17]:
for i in range(len(meta_rsna)):
    path = f'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Dataset\{meta_rsna.patient_id[i]}\{meta_rsna.view[i]}'
    # print(f'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Dataset\{meta_rsna.patient_id[i]}\{meta_rsna.view[i]}')
    if os.path.exists(path) == False:
        os.makedirs(path)
    if meta_rsna.Fake[i] == 1:
        # path1 = f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/archive/RSNA_1024x512/{meta_rsna.view[i]}_{meta_rsna.image_id[i]}.png'
        path2 = f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Dataset/{meta_rsna.patient_id[i]}/{meta_rsna.view[i]}/{meta_rsna.view[i]}_{meta_rsna.laterality[i]}_{meta_rsna.image_id[i]}.png'
        print(path2)
        # img = np.zeros((1024, 512), dtype=np.uint8)
        # cv2.imwrite(path2, img)
        # img = np.zeros((1024, 512), dtype=np.uint8)
        
        # # hiển thị bức ảnh
        # cv2.imshow("Black Image", img)
        # cv2.waitKey(0)
        
    

D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Dataset/10589/CC/CC_L_195400299.png
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Dataset/11094/MLO/MLO_L_1882170663.png
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Dataset/11094/MLO/MLO_L_1417771843.png
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Dataset/12153/CC/CC_L_1652537920.png
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Dataset/12725/MLO/MLO_R_1297379916.png
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Dataset/12725/MLO/MLO_R_1289574047.png
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Dataset/13116/MLO/MLO_R_238763713.png
D:/OneDrive - Industrial Universi

# Processing data Vindr

In [54]:
meta_vin = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\CSV_MAIN\vindr.csv')
meta_vin = meta_vin[["patient_id"	,"image_id",	"view"	,"laterality"	,"age"	,"cancer"]]
# meta_vin = meta_vin[meta_vin.cancer == 1]
# meta_vin.cancer

In [55]:
check_cancer_vin = {}
for i in range(len(meta_vin)):
    
    if meta_vin.image_id[i] in check_cancer_vin:
        if check_cancer_vin[meta_vin.image_id[i]] == 0:
            check_cancer_vin[meta_vin.image_id[i]] = meta_vin.cancer[i]
    else: check_cancer_vin.update({meta_vin.image_id[i]: meta_vin.cancer[i]})

In [56]:
# Fake = []
cnt = 0
# for i in range(len(meta_vin)):
#     if meta_vin.cancer[i] != check_cancer_vin[meta_vin.image_id[i]]:
#         Fake.append(1)
#         cnt+=1
#     else: Fake.append(0)
fake = []
for i in range(len(meta_vin)):
    # print(check_cancer_vin[meta_vin.image_id[i]])
    if meta_vin.cancer[i] != check_cancer_vin[meta_vin.image_id[i]]:
        fake.append(1)
        # print(meta_rsna[i])
        cnt+=1
    else:
        fake.append(0)

### Bộ vin không có tạo ảnh thêm

In [57]:
# "D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\VinDr-Mammo_Origin_image\VinDr-Mammo_Origin_image\CC_L\00a699bda0d1a1c2c7852322a7fdb23c.png"

In [58]:
from crop_image_for_you import *

In [59]:
meta_vin

,patient_id,image_id,view,laterality,age,cancer
0,10200894bf6f74b2ae92a0cecd37c60c,581b0e4d471a5c0adc6888cb038fa722,CC,R,59.0,1
1,10200894bf6f74b2ae92a0cecd37c60c,29896919b15b2dd978a36b2f68a5ad93,MLO,R,59.0,1
2,10200894bf6f74b2ae92a0cecd37c60c,8992f69db04084cbb4b3b31f6735e4eb,CC,L,59.0,0
3,10200894bf6f74b2ae92a0cecd37c60c,af0f6ad067fdf3e94c630ad4f5e5151f,MLO,L,59.0,0
4,450f7f9b2aa32754b076f0f53f0d8efc,33be94e3d81065693202c21aa662cfd2,CC,R,55.0,0
...,...,...,...,...,...,...
447,8ae2db86499819584e175cadf9099d25,8ec34cef1eb98d7c121509c8342b2212,MLO,L,53.0,0
448,3bdcc5d8c49f1e2b6332dc0a039ca35d,10e0f362333df810ac84a9db8fb3fd42,CC,R,34.0,1
449,3bdcc5d8c49f1e2b6332dc0a039ca35d,85a6579cbdc403cfc4dde0a8149ed855,MLO,R,34.0,1
450,3bdcc5d8c49f1e2b6332dc0a039ca35d,3cd51ee99070c4d625d52b848d5e9bfc,CC,L,34.0,0


In [60]:
pd1 = Process_Data(meta_vin)

pd1.my_method(list(range(len(meta_vin))))

## Processing data CMMD

In [63]:
cmmd = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\CSV_MAIN\Cmmd.csv')
cmmd = cmmd[["patient_id"	,"image_id",	"view"	,"laterality"	,"age"	,"cancer"]]
cmmd

,patient_id,image_id,view,laterality,age,cancer
0,D1-0045,D1-0045&1-2,CC,L,33,1
1,D1-0045,D1-0045&1-1,MLO,L,33,1
2,D1-0046,D1-0046&1-2,CC,R,43,1
3,D1-0046,D1-0046&1-1,MLO,R,43,1
4,D1-0047,D1-0047&1-2,CC,R,43,1
...,...,...,...,...,...,...
4289,D2-0748,D2-0748&1-1,MLO,R,29,0
4290,D2-0749,D2-0749&1-2,CC,L,42,1
4291,D2-0749,D2-0749&1-1,MLO,L,42,1
4292,D2-0749,D2-0749&1-4,CC,L,42,0


In [64]:
check_cancer_cmmd = {}
for i in range(len(cmmd)):
    
    if cmmd.image_id[i] in check_cancer_cmmd:
        if check_cancer_cmmd[cmmd.image_id[i]] == 0:
            check_cancer_cmmd[cmmd.image_id[i]] = cmmd.cancer[i]
    else: check_cancer_cmmd.update({cmmd.image_id[i]: cmmd.cancer[i]})

In [65]:
check_cancer_cmmd

{'D1-0045&1-2': 1,
 'D1-0045&1-1': 1,
 'D1-0046&1-2': 1,
 'D1-0046&1-1': 1,
 'D1-0047&1-2': 1,
 'D1-0047&1-1': 1,
 'D1-0051&1-2': 1,
 'D1-0051&1-1': 1,
 'D1-0053&1-2': 1,
 'D1-0053&1-1': 1,
 'D1-0055&1-2': 1,
 'D1-0055&1-1': 1,
 'D1-0056&1-2': 1,
 'D1-0056&1-1': 1,
 'D1-0058&1-2': 1,
 'D1-0058&1-1': 1,
 'D1-0059&1-2': 1,
 'D1-0059&1-1': 1,
 'D1-0061&1-2': 1,
 'D1-0061&1-1': 1,
 'D1-0063&1-2': 1,
 'D1-0063&1-1': 1,
 'D1-0064&1-2': 1,
 'D1-0064&1-1': 1,
 'D1-0067&1-2': 1,
 'D1-0067&1-1': 1,
 'D1-0068&1-2': 1,
 'D1-0068&1-1': 1,
 'D1-0070&1-2': 1,
 'D1-0070&1-1': 1,
 'D1-0071&1-2': 1,
 'D1-0071&1-1': 1,
 'D1-0075&1-2': 1,
 'D1-0075&1-1': 1,
 'D1-0078&1-2': 1,
 'D1-0078&1-1': 1,
 'D1-0084&1-2': 1,
 'D1-0084&1-1': 1,
 'D1-0085&1-2': 1,
 'D1-0085&1-1': 1,
 'D1-0087&1-4': 1,
 'D1-0087&1-3': 1,
 'D1-0087&1-2': 0,
 'D1-0087&1-1': 0,
 'D1-0089&1-4': 0,
 'D1-0089&1-3': 0,
 'D1-0089&1-2': 1,
 'D1-0089&1-1': 1,
 'D1-0091&1-2': 1,
 'D1-0091&1-1': 1,
 'D1-0101&1-2': 1,
 'D1-0101&1-1': 1,
 'D1-0102&1-

In [66]:
# Fake = []
cnt = 0
fake = []
for i in range(len(cmmd)):
    # print(check_cancer_vin[meta_vin.image_id[i]])
    if cmmd.cancer[i] != check_cancer_cmmd[cmmd.image_id[i]]:
        fake.append(1)
        # print(meta_rsna[i])
        cnt+=1
    else:
        fake.append(0)

In [68]:
cmmd['Fake'] = fake

In [69]:
cmmd

,patient_id,image_id,view,laterality,age,cancer,Fake
0,D1-0045,D1-0045&1-2,CC,L,33,1,0
1,D1-0045,D1-0045&1-1,MLO,L,33,1,0
2,D1-0046,D1-0046&1-2,CC,R,43,1,0
3,D1-0046,D1-0046&1-1,MLO,R,43,1,0
4,D1-0047,D1-0047&1-2,CC,R,43,1,0
...,...,...,...,...,...,...,...
4289,D2-0748,D2-0748&1-1,MLO,R,29,0,0
4290,D2-0749,D2-0749&1-2,CC,L,42,1,0
4291,D2-0749,D2-0749&1-1,MLO,L,42,1,0
4292,D2-0749,D2-0749&1-4,CC,L,42,0,0


In [70]:
cmmd_pro = pd.read_csv(r"D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\CSV_MAIN\Cmmd_external-1.csv")
cmmd_pro

,Unnamed: 0,path_img,patient_id,laterality,age,img_id,view,cancer
0,0,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D1-0001,R,44,D1-0001_1-1,MLO,0
1,1,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D1-0001,R,44,D1-0001_1-2,CC,0
2,2,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D1-0002,L,40,D1-0002_1-1,MLO,0
3,3,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D1-0002,L,40,D1-0002_1-2,CC,0
4,4,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D1-0003,L,39,D1-0003_1-2,CC,0
...,...,...,...,...,...,...,...,...
5585,5585,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D2-0748,R,29,D2-0748_1-2,CC,0
5586,5586,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D2-0749,L,42,D2-0749_1-2,CC,1
5587,5587,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D2-0749,L,42,D2-0749_1-4,CC,0
5588,5588,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D2-0749,L,42,D2-0749_1-3,MLO,0


In [73]:
cmmd_pro.img_id = cmmd_pro.img_id.apply(lambda x: x.replace('_', '&'))

In [75]:
cmmd_pro = cmmd_pro[['path_img', 'img_id']]

,path_img,img_id
0,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D1-0001&1-1
1,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D1-0001&1-2
2,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D1-0002&1-1
3,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D1-0002&1-2
4,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D1-0003&1-2
...,...,...
5585,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D2-0748&1-2
5586,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D2-0749&1-2
5587,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D2-0749&1-4
5588,/kaggle/input/cmmd2022/TheChineseMammographyDa...,D2-0749&1-3


In [77]:
path_dict = {}
path_dict.keys() = cmmd_pro.img_id

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (2711917290.py, line 2)

In [79]:
{cmmd_pro.img_id: cmmd_pro.path_img}

TypeError: unhashable type: 'Series'

In [92]:
path = []
for i in range(len(cmmd)):
    idimg = cmmd.image_id[i]
    paths = 'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/' + cmmd_pro[cmmd_pro.img_id == idimg].path_img.values[0].replace("/kaggle/input/cmmd2022/", "")
    path.append(paths)
    

In [94]:
cmmd['path_img'] = path

In [104]:
from crop_image_for_you import *

In [105]:
pds2 = Process_Data(cmmd)
pds2.my_method(list(range(len(cmmd))))

In [121]:
len(cmmd)

4294

# Processing data CDD-ESM

In [106]:
cdd = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\CSV_MAIN\cdd_cesm.csv')
cdd = cdd[["patient_id"	,"image_id",	"view"	,"laterality"	,"age"	,"cancer"]]
cdd

,patient_id,image_id,view,laterality,age,cancer
0,1,P1&L&DM&MLO,MLO,L,46,1
1,1,P1&L&DM&MLO,CC,L,46,0
2,3,P3&R&DM&CC,CC,R,45,0
3,3,P3&R&DM&MLO,MLO,R,45,0
4,3,P3&L&DM&CC,CC,L,45,1
...,...,...,...,...,...,...
633,323,P323&L&DM&MLO,MLO,L,45,1
634,326,P326&R&DM&CC,CC,R,51,0
635,326,P326&R&DM&MLO,MLO,R,51,0
636,326,P326&L&DM&CC,CC,L,51,1


In [108]:
len(cdd.patient_id.unique())

179

In [ ]:
'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/CDD-CESM/CDD-CESM/Low energy images of CDD-CESM/'

In [110]:
cdd.patient_id = cdd.patient_id.apply(lambda x: str(x)+"CDD")

In [115]:
cdd

,patient_id,image_id,view,laterality,age,cancer
0,1CDD,P1&L&DM&MLO,MLO,L,46,1
1,1CDD,P1&L&DM&MLO,CC,L,46,0
2,3CDD,P3&R&DM&CC,CC,R,45,0
3,3CDD,P3&R&DM&MLO,MLO,R,45,0
4,3CDD,P3&L&DM&CC,CC,L,45,1
...,...,...,...,...,...,...
633,323CDD,P323&L&DM&MLO,MLO,L,45,1
634,326CDD,P326&R&DM&CC,CC,R,51,0
635,326CDD,P326&R&DM&MLO,MLO,R,51,0
636,326CDD,P326&L&DM&CC,CC,L,51,1


In [118]:
from crop_image_for_you import *

In [119]:
pd4 = Process_Data(cdd)
pd4.my_method(list(range(len(cdd))))

In [120]:
cdd.to_csv(r"D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\CSV_MAIN\cdd_cesm_main.csv")

In [122]:
glob.glob(r"D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Dataset\*")

['D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Dataset\\01c3c13f0b852aed00b5147c21d0e649',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Dataset\\0246c6050121c176d2e8fcf91f22f926',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Dataset\\0f1a7b0d4efecb388a4b98ae5e2d5c29',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Dataset\\0fab8097f55f21d4bfa7667250122312',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Dataset\\10006',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Dataset\\10011',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Dataset\\10025',
 'D:\\OneDrive - Industrial Uni

In [123]:
len(glob.glob(r"D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Dataset\*"))

13507

# Create data meta

In [124]:
rsna = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\CSV_MAIN\rsna.csv')
vindr = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\CSV_MAIN\vindr.csv')
cmmd = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\CSV_MAIN\cmmd.csv')
cdd = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\CSV_MAIN\cdd_cesm.csv')

In [125]:
rsna = rsna[["patient_id"	,"image_id",	"view"	,"laterality"	,"age"	,"cancer"]]
vindr = vindr[["patient_id"	,"image_id",	"view"	,"laterality"	,"age"	,"cancer"]]
cmmd = cmmd[["patient_id"	,"image_id",	"view"	,"laterality"	,"age"	,"cancer"]]
cdd = cdd[["patient_id"	,"image_id",	"view"	,"laterality"	,"age"	,"cancer"]]


In [128]:
meta = pd.concat([rsna, vindr, cmmd, cdd])
meta = meta.reset_index()
meta = meta[["patient_id"	,"image_id",	"view"	,"laterality"	,"age"	,"cancer"]]
meta

,patient_id,image_id,view,laterality,age,cancer
0,10006,1874946579,CC,R,61.0,0
1,10006,1864590858,MLO,R,61.0,0
2,10006,462822612,CC,L,61.0,0
3,10006,1459541791,MLO,L,61.0,0
4,10011,541722628,CC,R,55.0,0
...,...,...,...,...,...,...
55555,323CDD,P323&L&DM&MLO,MLO,L,45.0,1
55556,326CDD,P326&R&DM&CC,CC,R,51.0,0
55557,326CDD,P326&R&DM&MLO,MLO,R,51.0,0
55558,326CDD,P326&L&DM&CC,CC,L,51.0,1


In [129]:
meta.to_csv(f'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\CSV_MAIN\meta_data.csv')

# Chuyển đổi thành 1 folder chứa nhiều ảnh

In [130]:
meta

,patient_id,image_id,view,laterality,age,cancer
0,10006,1874946579,CC,R,61.0,0
1,10006,1864590858,MLO,R,61.0,0
2,10006,462822612,CC,L,61.0,0
3,10006,1459541791,MLO,L,61.0,0
4,10011,541722628,CC,R,55.0,0
...,...,...,...,...,...,...
55555,323CDD,P323&L&DM&MLO,MLO,L,45.0,1
55556,326CDD,P326&R&DM&CC,CC,R,51.0,0
55557,326CDD,P326&R&DM&MLO,MLO,R,51.0,0
55558,326CDD,P326&L&DM&CC,CC,L,51.0,1


In [151]:
meta.image_id =meta.image_id.apply(lambda x: str(x).replace('&', '-'))

In [153]:
meta.to_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\meta_data.csv')

In [149]:
for id in range(len(meta)):
    file_name = f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset/{meta.patient_id[id]}/{meta.view[id]}/{meta.view[id]}_{meta.laterality[id]}_{meta.image_id[id]}.png'
    if '&' in file_name:
        path2 = file_name.replace('&', '-')
        # print(path2)
        os.rename(file_name, path2)


In [2]:
from remove_image import *

In [3]:
meta = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\meta_data.csv')


In [ ]:
with mp.Pool(12) as p:
    p.map(process_move, list(range(len(meta))))

In [147]:
for id in range(len(meta)):
    

KeyboardInterrupt: 

In [145]:
img = cv2.imread(r"D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Meta_image\CC_L_51088550.png")

In [146]:
img.shape

(1024, 512, 3)